In [1]:
library('ggplot2')
library('DESeq2')

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min



Attaching package: ‘S4Vectors’


The

In [16]:
countData <- read.csv('deseq2_input_raw_count.csv', header = TRUE, sep = ",")
head(countData)

,ensgene,gso32.BL6.N.N.4h.1,gso32.BL6.IFNb.N.4h.1,gso32.BL6.N.N.4h.2,gso32.BL6.IFNb.N.4h.2,gso32.BL6.N.N.24h.1,gso32.BL6.IFNb.N.24h.1,gso32.BL6.N.N.24h.2,gso32.BL6.IFNb.N.24h.2
,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,ENSMUSG00000033845,955,460,954,406,629,591,608,553
2,ENSMUSG00000025903,1163,849,1302,951,1098,1081,1073,1031
3,ENSMUSG00000033813,786,1291,894,1346,541,717,579,825
4,ENSMUSG00000033793,2541,2148,2598,1968,1606,1615,1549,1620
5,ENSMUSG00000025907,992,562,1096,603,1901,1288,1789,1374
6,ENSMUSG00000033740,106,123,122,137,453,546,474,613


In [18]:
metaData <- read.csv('deseq2_input_metadata.csv', header = TRUE, sep = ",")
metaData$time <- as.factor(metaData$time)
metaData$mouse <- as.factor(metaData$mouse)
metaData

id,treatment,time,mouse,group
<fct>,<fct>,<fct>,<fct>,<fct>
gso32.BL6.N.N.4h.1,Untreated,4,1,Untreated_4
gso32.BL6.IFNb.N.4h.1,IFNbeta,4,1,IFNbeta_4
gso32.BL6.N.N.4h.2,Untreated,4,2,Untreated_4
gso32.BL6.IFNb.N.4h.2,IFNbeta,4,2,IFNbeta_4
gso32.BL6.N.N.24h.1,Untreated,24,1,Untreated_24
gso32.BL6.IFNb.N.24h.1,IFNbeta,24,1,IFNbeta_24
gso32.BL6.N.N.24h.2,Untreated,24,2,Untreated_24
gso32.BL6.IFNb.N.24h.2,IFNbeta,24,2,IFNbeta_24


In [19]:
dds <- DESeqDataSetFromMatrix(countData=countData, 
                              colData=metaData, 
                              design=~mouse+group, tidy = TRUE)

In [20]:
dds

class: DESeqDataSet 
dim: 12532 8 
metadata(1): version
assays(1): counts
rownames(12532): ENSMUSG00000033845 ENSMUSG00000025903 ...
  ENSMUSG00000064368 ENSMUSG00000064370
rowData names(0):
colnames(8): gso32.BL6.N.N.4h.1 gso32.BL6.IFNb.N.4h.1 ...
  gso32.BL6.N.N.24h.2 gso32.BL6.IFNb.N.24h.2
colData names(5): id treatment time mouse group

# Wald

In [37]:
dds_ward_ifnb <- DESeq(dds, test="Wald")

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [39]:
write.csv(results(dds_ward_ifnb, contrast=c("group","IFNbeta_4","Untreated_4")), 'deseq2_output_wald_t4.csv')
write.csv(results(dds_ward_ifnb, contrast=c("group","IFNbeta_24","Untreated_24")), 'deseq2_output_wald_t24.csv')

In [42]:
summary(results(dds_ward_ifnb, contrast=c("group","IFNbeta_4","Untreated_4"), alpha = 0.05))


out of 12523 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 4442, 35%
LFC < 0 (down)     : 4327, 35%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [43]:
summary(results(dds_ward_ifnb, contrast=c("group","IFNbeta_24","Untreated_24"), alpha = 0.05))


out of 12523 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 3992, 32%
LFC < 0 (down)     : 4021, 32%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



# LRT

In [21]:
dds_lrt_ifnb <- DESeq(dds, test="LRT", reduced=~mouse)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [40]:
write.csv(results(dds_lrt_ifnb, contrast=c("group","IFNbeta_4","Untreated_4")), 'deseq2_output_lrt_t4.csv')
write.csv(results(dds_lrt_ifnb, contrast=c("group","IFNbeta_24","Untreated_24")), 'deseq2_output_lrt_t24.csv')

In [34]:
summary(results(dds_lrt_ifnb, contrast=c("group","IFNbeta_4","Untreated_4"), alpha = 0.05))


out of 12523 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 5549, 44%
LFC < 0 (down)     : 5737, 46%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [35]:
summary(results(dds_lrt_ifnb, contrast=c("group","IFNbeta_24","Untreated_24"), alpha = 0.05))


out of 12523 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 5591, 45%
LFC < 0 (down)     : 5693, 45%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

